In [1]:
# from SpatialTemporal_Diffusion import *
import numpy as np 
import torch
import torch.nn as nn
from torch import nn, einsum
from einops import rearrange, repeat, pack , unpack
import sys, os
module_path = os.path.abspath(os.path.join('..', 'einops_exts'))
if module_path not in sys.path:
    sys.path.append(module_path)
from einops_exts import check_shape, rearrange_many

In [2]:
optical_flow = np.load('/data/liaohx/voxelmorph/output_0718_7/warp.npy')
print(optical_flow.shape)

(10800, 2, 128, 128)


In [3]:
def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=32, num_channels=in_channels, eps=1e-6, affine=True)

class SpatialSelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x):  # 这里的输入是3帧
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b,c,h,w = q.shape
        q = rearrange(q, 'b c h w -> b (h w) c')
        k = rearrange(k, 'b c h w -> b c (h w)')
        w_ = torch.einsum('bij,bjk->bik', q, k)

        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = rearrange(v, 'b c h w -> b c (h w)')
        w_ = rearrange(w_, 'b i j -> b j i')
        h_ = torch.einsum('bij,bjk->bik', v, w_)
        h_ = rearrange(h_, 'b c (h w) -> b c h w', h=h)
        h_ = self.proj_out(h_)

        return x+h_

In [5]:
import torch
from einops import rearrange

# 假设 q_center 和 k_neighbour 是随机初始化的张量
b, c, t, h, w = 2, 3, 4, 5, 5
q_center = torch.randn(b, c, h, w)
k_neighbour = torch.randn(b, c, t, h, w)

# 重新排列张量
q_center = rearrange(q_center, 'b c h w -> b (h w) c')
k_neighbour = rearrange(k_neighbour, 'b c t h w -> b c (h w) t')

# 检查形状
print(q_center.shape)  # 应该是 (b, h*w, c)
print(k_neighbour.shape)  # 应该是 (b, c, h*w, t)

# 计算注意力权重
w_ = torch.einsum('bic, bcht -> biht', q_center, k_neighbour)

# 检查结果形状
print(w_.shape)  # 应该是 (b, h*w, h*w, t)

torch.Size([2, 25, 3])
torch.Size([2, 3, 25, 4])
torch.Size([2, 25, 25, 4])


In [ ]:
def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=32, num_channels=in_channels, eps=1e-6, affine=True)

class SpatialSelfAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x_center, x_neighbour, x):
        # the input shape of x should be (B, C, t, h w)  
        B, C, t, h, w = x.shape
        # to find the neighbour tensor for each frame
        for frame in range(x.shape[2]):
            
        
        
        b1, c, h, w = x_center.shape
        b3, c, t, h, w = x_neighbour.shape
        
        h_center = x_center
        h_neighbour = x_neighbour
        
        h_center = self.norm(h_center)
        h_neighbour = self.norm(h_neighbour)
        
        q_center = self.q(h_center)  # b1, c, h, w
        
        k_neighbour = self.k(h_neighbour)  # b3, c, h, w
        v_neighbour = self.v(h_neighbour)  # b3, c, h, w
        
        # 将 q_center 和 k_neighbour 沿着第一个维度进行广播再乘到一起去
        q_center = rearrange(q_center, 'b c h w -> b (h w) c')
        k_neighbour = rearrange(k_neighbour, 'b c t h w -> b c (hw) t')
        
        # 沿着时间维度进行广播再计算attention操作
        w_ = torch.einsum('bij, bjkt -> bikt', q_center, k_neighbour)  # the shape should be like (b hw hw t)
        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)  
        # attend to values
        v_ = rearrange(v_neighbour, 'b c t h w -> b c (h w) t')
        w_ = rearrange(w_, 'b i j t -> b j i t')
        
        h_ = torch.einsum('bijt, bjkt -> bikt', v_, w_)  # b c hw t \times b hw hw t -> b c hw t
        h_ = rearrange()
        
        
        h_ = self.norm(h_) 
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b,c,h,w = q.shape
        q = rearrange(q, 'b c h w -> b (h w) c')
        k = rearrange(k, 'b c h w -> b c (h w)')
        w_ = torch.einsum('bij,bjk->bik', q, k)  # b (hw) c \times b c (hw) -> b (hw) (hw)

        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = rearrange(v, 'b c h w -> b c (h w)')
        w_ = rearrange(w_, 'b i j -> b j i')  # swap two axis??  
        
        h_ = torch.einsum('bij,bjk->bik', v, w_)
        h_ = rearrange(h_, 'b c (h w) -> b c h w', h=h)
        h_ = self.proj_out(h_)

        return x+h_

## Temporal Data Consistency Layer

In [6]:
# enhance the temporal consistency for denoised initial prediction
class TemporalConsistencyBlock(nn.Module):
    """docstring for TemporalConsistencyBlock."""
     def __init__(self):
        super().__init__()
        pass